In [0]:
######## Install Tensorflow version 1.13.2
!pip install tensorflow==1.13.2 
!pip3 install flask-ngrok

In [0]:
######## Official git repo
##
##     https://www.github.com/purvesh-linux11/imageCaptioning.git
##
##### clone project moduls

!git clone https://www.github.com/purvesh-linux11/imageCaptioning.git
!cp -r /content/imageCaptioning/* /content/

In [0]:
##### copy model from drive and extract here

!cp -r /content/drive/'My Drive'/MODELS/image_captioning_model.zip /content/  
!unzip image_captioning_model.zip
!cp -r /content/image_captioning_model/* /content/

In [0]:
##### copy photos from drive

!cp -r /content/drive/'My Drive'/photos/* /content/photos/

In [0]:
#############################################################
##
##
## Download Model and place it current directory 
##                  (Link in Readme file)
##
##############################################################

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
from os import listdir, path

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import PIL.Image
import PIL.ImageOps
import PIL.ImageFont
import PIL.ImageDraw
import textwrap
from io import BytesIO

from im2txt import configuration
from im2txt import inference_wrapper
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary

In [0]:
####### create directories for store usr uploads
!mkdir -p /content/user_img/
!mkdir -p /content/user_zip/
!mkdir -p /content/user_download/

In [0]:
#########  send generated logs to download directory
def send_logs():
  !mv /content/results/logs.txt /content/user_download/
  return

In [0]:
######  clean usr directories to new uploads
def clear_dir():
  !rm -rf /content/photos/*
  !rm -rf /content/results/*

In [0]:
# Choose the trained model --> current is 2
clear_dir()   ##### remove git example
model_number = "2"
model_path = "/content/model.ckpt-"+model_number+"000000"   # Give model path
vocab_path = "/content/word_counts"+model_number+".txt"     # Give word_counts file path
tf.logging.set_verbosity(tf.logging.INFO)

# Build the inference graph.
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(), model_path)
g.finalize()

# Create the vocabulary.
vocab = vocabulary.Vocabulary(vocab_path) 

#######################################################
### if tensorflow version is <1.13.2 then you have to check variables name as per tensorflow version

OLD_CHECKPOINT_FILE = "/content/model.ckpt-2000000"
NEW_CHECKPOINT_FILE = "/content/model.ckpt-2000000"

import tensorflow as tf
vars_to_rename = {
    "lstm/BasicLSTMCell/Linear/Matrix": "lstm/basic_lstm_cell/kernel",
    "lstm/BasicLSTMCell/Linear/Bias": "lstm/basic_lstm_cell/bias",
}
new_checkpoint_vars = {}
reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
for old_name in reader.get_variable_to_shape_map():
  if old_name in vars_to_rename:
    new_name = vars_to_rename[old_name]
  else:
    new_name = old_name
  new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

init = tf.global_variables_initializer()
saver = tf.train.Saver(new_checkpoint_vars)

with tf.Session() as sess:
  sess.run(init)
  saver.save(sess, NEW_CHECKPOINT_FILE)

#######################################################

sess = tf.Session(graph=g)
# Load the model from checkpoint.
restore_fn(sess)

# Prepare the caption generator. Here we are implicitly using the default
# beam search parameters. See caption_generator.py for a description of the
# available beam search parameters.
generator = caption_generator.CaptionGenerator(model, vocab)

In [0]:
def generator_caption():
    image_path = "./photos/"    #### provide path where image is stored
    filename = listdir(image_path)
    filenames = [f for f in filename if '.jpg' in f or '.png' in f or '.jpeg' in f]
    store = open('/content/results/logs.txt','w')   #### directory to store captions file

    for file in filenames:
        try:
            img = PIL.Image.open(image_path+file).convert('RGBA')
            box = PIL.Image.new('RGBA', img.size, (255,255,255,0))
            draw = PIL.ImageDraw.Draw(box)
            image = open(image_path+file,'rb').read() # Read the image as bytes
            captions = generator.beam_search(sess, image)
            for i, caption in enumerate(captions):
                # Ignore begin and end words.
                sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
                sentence = " ".join(sentence)
                if i==0:
                  print(file+" : %s " % (sentence))
                  #create log file image_name--image_caption
                  store.write(file+" : "+sentence+"\n")
        except KeyboardInterrupt:
            store.close()
            break
    store.close()
    send_logs()
    return

# **Flask Server setup**

In [0]:
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
from flask import send_from_directory
from flask import Flask

In [0]:
UPLOAD_USER_IMG = '/content/user_img'
UPLOAD_USER_ZIP = '/content/user_zip'
DOWNLOAD_FILE = '/content/user_download'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
ALLOWED_ZIP = {'zip','tar'}

In [0]:
#### move contents from user_img to photos
def move_img():
  !cp -r /content/user_img/* /content/photos/
  !rm -rf /content/user_img/*

In [0]:
##### extract zip into photos directory
def move_zip():
  !unzip /content/user_zip/* -d /content/photos/
  !rm -rf /content/user_zip/*

In [0]:
app = Flask(__name__)
run_with_ngrok(app)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def allowed_zip(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_ZIP


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    app.config['UPLOAD_USER_IMG'] = UPLOAD_USER_IMG
    app.config['UPLOAD_USER_ZIP'] = UPLOAD_USER_ZIP
    app.config['DOWNLOAD_FILE'] = DOWNLOAD_FILE
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_zip(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_USER_ZIP'], filename))
            move_zip()
            generator_caption()
            clear_dir()
            return send_from_directory(app.config['DOWNLOAD_FILE'],'logs.txt',as_attachment=True)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_USER_IMG'], filename))
            move_img()
            generator_caption()
            clear_dir()
            return send_from_directory(app.config['DOWNLOAD_FILE'],'logs.txt')
    """
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload><br/>
      <h3>Wait some moment to generate caption.</h3>
    </form>
    '''
    """
    return '''
    <!DOCTYPE html>
<html>
  <head>
    <title>image captioning</title>   
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.4.1/css/all.css" integrity="sha384-5sAR7xN1Nv6T6+dT2mhtzEpVJvfS3NScPQTrOxhwjIuvcA67KV2R5Jz6kr4abQsz" crossorigin="anonymous">
    <style>
      html, body {
      display: flex;
      justify-content: center;
      align-items: center;
      height: 100%;
      }
      body, div, p { 
      padding: 0;
      margin: 0;
      outline: none;
      font-family: Roboto, Arial, sans-serif;
      font-size: 16px;
      color: #666;
      }
      body {
	  background-image: url('https://images.unsplash.com/uploads/1412825195419af52b492/8bc72ed7?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=751&q=80');
  background-repeat: no-repeat;
     
      background-size:cover;
      }
      h1 {
      margin: 0 0 10px 0;
      font-weight: 400;
      }
      .main-block {
      display: flex;
      width: 90%;
      padding: 10px 0; 
      border-radius: 5px;
      box-shadow: 1px 1px 8px 0px #666; 
      background: #fff;
      }
      .block-item {
      width: 50%;
      padding: 20px; 
      }
      .block-item.right {
      border-left: 1px solid #aaa;
      }
      i {
      width: 50px;
      font-size: 24px;
      }
    
      }
     
    </style>
  </head>
  <body>
    <div class="main-block">
	
      <div class="block-item left">
	  <form method = post enctype=multipart/form-data>
        <h1>Upload your file here:</h1>
        <p><small>To get your caption related to image file you provided.</small>
        </p>
      </div>
      <div class="block-item right">
     <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload><br/>
    </form>
      </div>
      </div>
    </div>
  </body>
</html>
    '''

app.run()